In [42]:
import config
import preprocessing  as p
import postprocessing as m

In [43]:
server='/Volumes/EEG Core HD/Pipeline For Olga/Pipeline Example Files/Mats/'
path='/Users/Olga/CS_projects/SWISC/package/datanpy_newest_scored/Feats_Fourier_and_PSD'
path_1='/Volumes/EEG Core HD/Pipeline For Olga/Pipeline Example Files/Mats/NPM661-664'
path_2='/Volumes/EEG Core HD/Pipeline For Olga/Pipeline Example Files/Mats/NPM661-664/SlSz'

#imports single file
path_3='/Volumes/EEG Core HD/Pipeline For Olga/Pipeline Example Files/Mats/NPM661-664/SlSz/NPM661-664 210711 070952_053.smrx m1 NPM661 SlSz.mat'

In [44]:
p.return_file_list_from_server(path_3)

['/Volumes/EEG Core HD/Pipeline For Olga/Pipeline Example Files/Mats/NPM661-664/SlSz/NPM661-664 210711 070952_053.smrx m1 NPM661 SlSz.mat']

In [45]:
p.extract_metadata(path_3)

{'Cohort': 'NPM661-664',
 'Mouse': 'm1',
 'Annotation Type': 'SlSz',
 'Date': '210711',
 'Time': '070952_053'}

In [46]:
dict=p.download_new_file(path_3)


In [47]:
dict.keys()

dict_keys(['V1_ECog', 'V1_EMG', 'V1_HPC_L', 'V1_HPC_R', 'V1_NPM661slslsl', 'V1_NPM661szsz'])

In [48]:
d_dict=p.decimate_all_channels(dict)

In [49]:
ordered_array=p.create_epochs(d_dict)
    

In [50]:
X_test=p.feature_generation(ordered_array)

In [51]:
X_test.shape

(2160, 100)

In [52]:
dict.keys()

dict_keys(['V1_ECog', 'V1_EMG', 'V1_HPC_L', 'V1_HPC_R', 'V1_NPM661slslsl', 'V1_NPM661szsz'])

In [53]:
import re
import numpy as np

In [54]:
for k in dict.keys():
        pattern = r'(?i)sl'
        match = re.search(pattern, k)
        if match:
            print("Found:", match.group())
            # print(np.array(new_dict[k]['codes'][0]).shape)
            Sleep_codes=np.array(dict[k]['codes'][0])
            # epoch_ratio=config.target_epoch_count//len(Sleep_codes)
            # sleepEpochsRound=config.target_epoch_count//epoch_ratio       
            a_reshaped =Sleep_codes[:config.target_epoch_count, 0].reshape(config.target_epoch_count, 1).flatten()

Found: sl


In [55]:
a_reshaped.shape

(2160,)

In [56]:
y_test=a_reshaped

In [57]:
y_test.shape

(2160,)

In [58]:
X_predict=m.generate_x_sequences(X_test, 3)
y_predict=m.generate_y_sequences(y_test, 3)

In [59]:
from keras.models import load_model

import numpy as np
import seaborn as sns
import matplotlib as mpl
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, log_loss, cohen_kappa_score, matthews_corrcoef
import pandas as pd
import os
import config
from sklearn.preprocessing import StandardScaler
from keras.models import clone_model

In [60]:
from keras.models import load_model
model=load_model('/Users/Olga/CS_projects/SWISC/package/Models/SWISC 4 Channel and RMS.h5')

In [61]:
predictions = model.predict(X_predict)
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(y_predict, axis=1)


accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
kappa = cohen_kappa_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)

overview={
    'File Name': path.split('/')[-1],
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1,
    'Cohen Kappa': kappa,
    'MCC': mcc,
    # 'Log Loss': loss
}
    # Return as a dictionary
    

68/68 [==============================] - 2s 9ms/step


/Users/Olga/anaconda3/envs/brandon_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Olga/anaconda3/envs/brandon_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
overview

{'File Name': 'Feats_Fourier_and_PSD',
 'Accuracy': 0.20009285051067782,
 'Precision': 0.22382478632478633,
 'Recall': 0.0918520337514751,
 'F1 Score': 0.07114463330778863,
 'Cohen Kappa': -0.0135418672194072,
 'MCC': -0.019414424689084758}